In [29]:
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, roc_curve

In [30]:
# GPU 1번 슬롯만 사용하도록 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [31]:
# GPU 디바이스 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가 방식으로 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    
    except RuntimeError as e:
        print(e)

In [32]:
def scaled_dot_product_attention(query, key, value, mask=None):
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    d_k = tf.cast(tf.shape(key)[-1], tf.float32) #Float32로 형 변환
    scaled_attention_logits = matmul_qk / tf.math.sqrt(d_k)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, value)
    return output, attention_weights

In [33]:
# Multi-Head Attention
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x):
        x = tf.reshape(x, (tf.shape(x)[0], -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, x, mask=None):
        q = self.split_heads(self.wq(x))
        k = self.split_heads(self.wk(x))
        v = self.split_heads(self.wv(x))
        attention_output, _ = scaled_dot_product_attention(q, k, v, mask)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        attention_output = tf.reshape(attention_output, (tf.shape(attention_output)[0], -1, self.d_model))
        return self.dense(attention_output)

In [34]:
# Position-wise Feed-Forward Network
class PositionwiseFeedforward(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedforward, self).__init__()
        self.dense1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.dense2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        return self.dense2(self.dense1(x))

In [35]:
# Positional Encoding
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.positional_encoding = self.positional_encoding(max_len, d_model)

    def positional_encoding(self, max_len, d_model):
        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        pos_enc = np.zeros((max_len, d_model))
        pos_enc[:, 0::2] = np.sin(position * div_term)
        pos_enc[:, 1::2] = np.cos(position * div_term)
        return tf.constant(pos_enc, dtype=tf.float32)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.positional_encoding[:seq_len, :]

In [36]:
# Transformer Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedforward(d_model, d_ff)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, mask=None, training=False):
        attn_output = self.attention(x, mask)
        attn_output = self.dropout(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2

In [37]:
# Transformer Model
class TransformerModel(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, d_ff, max_len, num_classes, dropout_rate=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = tf.keras.layers.Dense(d_model)
        self.positional_encoding = PositionalEncoding(max_len, d_model)
        self.transformer_blocks = [TransformerBlock(d_model, num_heads, d_ff, dropout_rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.final_layer = tf.keras.layers.Dense(num_classes, activation = 'sigmoid') #이진분류

    def call(self, x, mask=None, training=False):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        for block in self.transformer_blocks:
            x = block(x, mask, training=training)
        x = self.dropout(x, training=training)
        return self.final_layer(x)

In [38]:
# 데이터 로딩 및 전처리
def load_npy_files(folder_path):
    data = []
    labels = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(folder_path, file_name)
            data.append(np.load(file_path))
            labels.append(file_name.split('_')[0])  # 파일명에서 라벨 추출 (예시)
    return np.array(data), np.array(labels)

In [39]:
def split_and_save_data(data, labels, save_path, split_ratio=(0.8, 0.1, 0.1)):
    # Split data into train, validation, and test sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(
        data, labels, test_size=1 - split_ratio[0], stratify=labels
    )
    valid_data, test_data, valid_labels, test_labels = train_test_split(
        temp_data, temp_labels, test_size=split_ratio[2] / (split_ratio[1] + split_ratio[2]), stratify=temp_labels
    )
    
    # Create directories
    os.makedirs(os.path.join(save_path, 'train'), exist_ok=True)
    os.makedirs(os.path.join(save_path, 'valid'), exist_ok=True)
    os.makedirs(os.path.join(save_path, 'test'), exist_ok=True)

    # Save splits
    np.save(os.path.join(save_path, 'train', 'data.npy'), train_data)
    np.save(os.path.join(save_path, 'train', 'labels.npy'), train_labels)
    np.save(os.path.join(save_path, 'valid', 'data.npy'), valid_data)
    np.save(os.path.join(save_path, 'valid', 'labels.npy'), valid_labels)
    np.save(os.path.join(save_path, 'test', 'data.npy'), test_data)
    np.save(os.path.join(save_path, 'test', 'labels.npy'), test_labels)

In [40]:
def load_and_process_data(normal_folder, vf_folder, save_path):
    normal_data = []
    normal_labels = []
    vf_data = []
    vf_labels = []

    # Process normal data
    for file_name in os.listdir(normal_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(normal_folder, file_name)
            normal_data.append(np.load(file_path))
            normal_labels.append(0)  # Label for normal data

    # Process VF data
    for file_name in os.listdir(vf_folder):
        if file_name.endswith('.npy'):
            label = int(file_name.split('_')[0])  # Extract label from file name
            file_path = os.path.join(vf_folder, file_name)
            vf_data.append(np.load(file_path))
            vf_labels.append(1)  # Label for VF data

    normal_data = np.array(normal_data)
    normal_labels = np.array(normal_labels)
    vf_data = np.array(vf_data)
    vf_labels = np.array(vf_labels)

    # Combine data
    combined_data = np.concatenate([normal_data, vf_data], axis=0)
    combined_labels = np.concatenate([normal_labels, vf_labels], axis=0)

    # Split and save
    split_and_save_data(combined_data, combined_labels, save_path)

In [41]:
def load_dataset(folder_path, batch_size, shuffle_buffer_size=1000, repeat=False):
    data = np.load(os.path.join(folder_path, 'data.npy'))
    labels = np.load(os.path.join(folder_path, 'labels.npy'))
    
    # NaN 값이 있는 인덱스 찾기
    nan_indices = np.any(np.isnan(data), axis=1)  # axis=1은 시퀀스 차원 기준

    # NaN 값을 포함하는 인덱스
    nan_indices = np.where(nan_indices)[0]

    # NaN 값을 포함하는 인덱스를 제외하기
    mask = np.ones(data.shape[0], dtype=bool)
    mask[nan_indices] = False
    
    data_cleaned = data[mask]
    labels_cleaned = labels[mask]
    
    # 레이블을 (batch_size, 1) 형태로 변환
    labels_cleaned = np.expand_dims(labels_cleaned, axis=-1)

    dataset = tf.data.Dataset.from_tensor_slices((data_cleaned, labels_cleaned))
    
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    
    dataset = dataset.batch(batch_size)
    
    if repeat:
        dataset = dataset.repeat()
    
    return dataset

In [86]:
#예측 시간 입력

pred_min = 60
if pred_min == 30:
    vf_folder = '/smc_work/datanvme/VF/vf_before_min_30_60/'
elif pred_min > 30:
    vf_folder = '/smc_work/datanvme/VF/vf_before_min_60_120/'
elif pred_min > 60:
    vf_folder = '/smc_work/datanvme/VF/vf_before_min_120_180/'

In [87]:
vf_folder

'/smc_work/datanvme/VF/vf_before_min_60_120/'

In [15]:
# Paths
normal_folder = '/smc_work/datanvme/VF/normal/'
save_path = '/smc_work/datanvme/VF/split_data/'

In [42]:
normal_folder = '/smc_work/datanvme/VF/vf_before_min_120_180/' # 60 min ~ 120 min
vf_folder = '/smc_work/datanvme/VF/vf_before_min_30_60/' # 30 min ~ 60 min

In [43]:
# Execute data processing
load_and_process_data(normal_folder, vf_folder, save_path)

In [44]:
def load_labels(folder_path):
    labels = np.load(os.path.join(folder_path, 'labels.npy'))
    print("Loaded labels shape:", labels.shape)
    print("Sample labels:", labels[:10])  # 처음 10개 레이블 출력

In [45]:
# 모델 하이퍼파라미터
num_layers = 4
d_model = 128
num_heads = 4
d_ff = 512
max_len = 2560  # 시퀀스 길이
num_classes = 1  
dropout_rate = 0.1
batch_size = 32

In [46]:
train_dataset = load_dataset(os.path.join(save_path, 'train'), batch_size)
valid_dataset = load_dataset(os.path.join(save_path, 'valid'), batch_size, repeat=False)
test_dataset = load_dataset(os.path.join(save_path, 'test'), batch_size, repeat=False)

In [47]:
# 모델 정의 및 컴파일
optimizer = Adam(learning_rate=1e-4)  # 기존보다 더 낮은 학습률로 설정
model = TransformerModel(num_layers, d_model, num_heads, d_ff, max_len, num_classes, dropout_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
# 모델 저장 경로 생성
model_save_path = f'/smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel{d_model}_heads{num_heads}_layers{num_layers}_dropout{dropout_rate:.1f}'

# 폴더 생성
os.makedirs(model_save_path, exist_ok=True)

In [49]:
model_save_path

'/smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1'

In [50]:
# ModelCheckpoint 콜백 정의
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(model_save_path, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.4f}'),  # 모델 저장 경로
    monitor='val_loss',  # 모니터링할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # 최소값을 기준으로 저장
    verbose=1  # 로그 출력
)

In [51]:

# 조기 종료 콜백 정의
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # 모니터링할 지표
    patience=3,  # patience는 몇 epoch동안 개선이 없을 때 종료할 지점
    mode='min',  # 최소값을 기준으로 조기 종료
    verbose=1  # 로그 출력
)

In [52]:
# 모델 학습
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=valid_dataset,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/10
110/110 [==============================] - ETA: 0s - loss: 0.7259 - accuracy: 0.5701
Epoch 00001: val_loss improved from inf to 0.67209, saving model to /smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1/model_epoch_01_val_loss_0.6721


INFO:tensorflow:Assets written to: /smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1/model_epoch_01_val_loss_0.6721/assets


INFO:tensorflow:Assets written to: /smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1/model_epoch_01_val_loss_0.6721/assets


110/110 [==============================] - 102s 892ms/step - loss: 0.7259 - accuracy: 0.5701 - val_loss: 0.6721 - val_accuracy: 0.6181
Epoch 2/10
110/110 [==============================] - ETA: 0s - loss: 0.6841 - accuracy: 0.5938
Epoch 00002: val_loss improved from 0.67209 to 0.66840, saving model to /smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1/model_epoch_02_val_loss_0.6684


INFO:tensorflow:Assets written to: /smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1/model_epoch_02_val_loss_0.6684/assets


INFO:tensorflow:Assets written to: /smc_work/home/weladmin/Desktop/code/Research/shea/VF/models/transformer_dmodel128_heads4_layers4_dropout0.1/model_epoch_02_val_loss_0.6684/assets


110/110 [==============================] - 98s 889ms/step - loss: 0.6841 - accuracy: 0.5938 - val_loss: 0.6684 - val_accuracy: 0.6186
Epoch 3/10
110/110 [==============================] - ETA: 0s - loss: 0.6754 - accuracy: 0.6088
Epoch 00003: val_loss did not improve from 0.66840
110/110 [==============================] - 90s 820ms/step - loss: 0.6754 - accuracy: 0.6088 - val_loss: 0.6687 - val_accuracy: 0.6186
Epoch 4/10
110/110 [==============================] - ETA: 0s - loss: 0.6725 - accuracy: 0.6095
Epoch 00004: val_loss did not improve from 0.66840
110/110 [==============================] - 90s 822ms/step - loss: 0.6725 - accuracy: 0.6095 - val_loss: 0.6908 - val_accuracy: 0.6186
Epoch 5/10
110/110 [==============================] - ETA: 0s - loss: 0.6730 - accuracy: 0.6160
Epoch 00005: val_loss did not improve from 0.66840
110/110 [==============================] - 90s 822ms/step - loss: 0.6730 - accuracy: 0.6160 - val_loss: 0.6703 - val_accuracy: 0.6186
Epoch 00005: early stop

In [ ]:
from PIL import Image
# 이미지를 저장할 함수 정의
def save_image_with_dpi(image_array, file_path, dpi=(1200, 1200)):
    # NumPy 배열을 이미지로 변환
    image = Image.fromarray(image_array.astype('uint8'))

    # 이미지 저장 시 DPI 설정
    image.save(file_path, dpi=dpi)

# 예시: ROC 곡선을 그린 후 이미지 저장
def plot_and_save_roc_curve(fpr, tpr, auc, file_path):
    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')

    # 현재 플롯을 이미지로 저장
    plt.savefig(file_path, dpi=(1200, 1200))
    plt.close()

# 예시 데이터 (실제 사용 시 fpr, tpr, auc 값을 넣으세요)
fpr = np.linspace(0, 1, 100)
tpr = np.linspace(0, 1, 100)
auc = 0.95

# ROC 곡선을 그린 후 1200 DPI로 이미지 저장
plot_and_save_roc_curve(fpr, tpr, auc, str(pred_min) + '_min_roc_curve.png')

In [84]:
# 테스트 데이터에서 예측
def predict_and_evaluate(model, test_dataset):
    # 모델을 평가 모드로 설정
    model.evaluate(test_dataset)  # 정확도 출력
    y_true = []
    y_pred = []

    for x_batch, y_batch in test_dataset:
        y_true.extend(y_batch.numpy().flatten())
        y_pred.extend(model.predict(x_batch).flatten())
        
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # AUC 계산
    auc = roc_auc_score(y_true, y_pred)
    print(f"Test AUC: {auc:.4f}")

    # ROC 곡선 계산
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    
    # ROC 곡선 시각화
    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# 모델 평가 및 AUC 시각화
predict_and_evaluate(model, test_dataset)

252/252 [==============================] - 74s 294ms/step - loss: 0.0173 - accuracy: 0.9946


In [ ]:
# 모델의 중간 출력을 확인하는 코드
class DebugModel(tf.keras.Model):
    def __init__(self, original_model):
        super(DebugModel, self).__init__()
        self.model = original_model

    def call(self, x, mask=None, training=False):
        x = self.model.embedding(x)
        print(f"Embedding output shape: {x.shape}")  # 디버깅
        x = self.model.positional_encoding(x)
        print(f"After positional encoding shape: {x.shape}")  # 디버깅
        for block in self.model.transformer_blocks:
            x = block(x, mask, training=training)
            print(f"After transformer block shape: {x.shape}")  # 디버깅
        x = self.model.dropout(x, training=training)
        print(f"After dropout shape: {x.shape}")  # 디버깅
        x = self.model.final_layer(x)
        print(f"Final output shape: {x.shape}")  # 디버깅
        return x

debug_model = DebugModel(model)